In [ ]:
### SET DRIVE PATH ###
drive = "/SETDRIVEPATH/data_lsd_trip/"

In [ ]:
import nrrd
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [ ]:
## loading and processing validation phantom data
path = drive + "data_paus/single_tubes_transversal/"
pa, nrrd_options = nrrd.read(path +"rCu_tube0_bg0_BMode_data_pa.nrrd")
data = np.zeros([pa.shape[0], pa.shape[1], pa.shape[2], 5, 5])
rCusBG = [0,25,50,75,100]
rCus = [0,25,50,75,100]
for i_rCu, rCu in enumerate(rCus):    
    for i_rCuBG, rCuBG in enumerate(rCusBG):
        data[..., i_rCu, i_rCuBG], nrrd_options = nrrd.read(path +
            "rCu_tube"+ str(rCu) +"_bg"+ str(rCuBG) +"_BMode_data_pa.nrrd")
avg_pa = np.zeros([pa.shape[0], pa.shape[1], 64, 5, 5])
for i in range(64):
    avg_pa[..., i, :, :] = np.mean(data[:,:, i::64, :, :], axis=2)
    
## cropping the tube/roi by selecting a patch 10px around the maximum intensity pixel
roi = np.asarray([44, 60])
roi_size = 40
patch_radius = 10
center_tube = np.zeros([2*patch_radius, 2*patch_radius, avg_pa.shape[2], len(rCus), len(rCusBG)])
tube_pos = np.zeros([2, len(rCus), len(rCusBG)])

for i_rCu in range(len(rCus)):
    for i_rCuBG in range(len(rCusBG)):
        _image = np.mean(avg_pa[roi[0]:roi[0]+roi_size, roi[1]:, :, i_rCu, i_rCuBG], axis=2)
        max_pos = np.unravel_index(np.argmax(_image, axis=None), _image.shape) + roi
        tube_pos[:, i_rCu, i_rCuBG] = max_pos
        center_tube[... , i_rCu, i_rCuBG] = avg_pa[max_pos[0]-patch_radius:max_pos[0]+patch_radius, 
                                                   max_pos[1]-patch_radius:max_pos[1]+patch_radius, 
                                                   :, i_rCu, i_rCuBG]
## pulse energy correction
df_opo = pd.read_csv(
    "/media/thomas/extSSD/data_lsd_trip/data_calibration/opo_energy_spectrum.csv", header=0)
df_sulfates = pd.read_csv(
    "../spectrophotometer/output_spectra.csv", header=0)
_nm = np.arange(680,981,20)
df_sulfates = df_sulfates[df_sulfates["lambda/nm"].isin(_nm)]
mean_test_corr = np.zeros([avg_pa.shape[0], avg_pa.shape[1], 16, 4, 
                           avg_pa.shape[3], avg_pa.shape[4]])

corr_opo = np.zeros([1, 1, df_opo['mean E after fiber/mJ'].size])
corr_opo[0,0,:] = np.asarray(df_opo['mean E after fiber/mJ'])

for i_rCu in range(len(rCus)):
    for i_rCuBG in range(len(rCusBG)):
        for i_ill in [0,1,2,3]:
            mean_test_corr[:, :, :, i_ill, i_rCu, i_rCuBG] = avg_pa[:, :, 
                                                                    16*i_ill:16*(i_ill+1), 
                                                                    i_rCu, i_rCuBG]/corr_opo
## loading "digital twin"
in_silico_path = drive + "data_in_silico/LSD_TRIP_exp01/LSD_TRIP_baseline_gelpadiswater_rCu_descriptor_data.csv"
df_in_silico = pd.read_csv(in_silico_path)

In [ ]:
## quick and dirty Linear Unmixing implementation
initial_guess_rCu = 0
initial_guess_tMix = 0
stop = 16

x0 = np.array([initial_guess_rCu, initial_guess_tMix], dtype=float)

#coeffs[]: rCu, tMix
def res(coeffs, wavelength_measured, spectrum_measured):
    return spectrum_measured - (coeffs[0] * coeffs[1] * np.asarray(df_sulfates["mua(Cu)//cm"])[::-1][:stop] + 
                 (coeffs[1] - coeffs[0]*coeffs[1]) * np.asarray(df_sulfates["mua(Ni)//cm"])[::-1][:stop])    
measured_wavelengths = np.asarray(df_sulfates["lambda/nm"])

for i_rCuBG in range(len(rCusBG)):
    for i_rCu in range(len(rCus)):
        for i_ill in [0]:
            measured_spectrum = np.max(mean_test_corr[50:70,75:83,:stop,i_ill,i_rCu,i_rCuBG], axis=(0,1))
            fit = least_squares(res, x0, args=((measured_wavelengths, measured_spectrum)))

In [ ]:
matplotlib.rcParams.update({'font.size': 7})
matplotlib.rc('xtick', labelsize=7)
matplotlib.rc('ytick', labelsize=7)
fig, ax = plt.subplots(1,2, figsize=(6.5,2), dpi=600)
cmap = matplotlib.cm.get_cmap('viridis')
d = 10

for i_rCu in [4]:
    for i_rCuBG in [0,4]:
        _tmp = np.max(mean_test_corr[int(tube_pos[0, i_rCu, i_rCuBG])-9:int(tube_pos[0, i_rCu, i_rCuBG])+9,
                                     int(tube_pos[1, i_rCu, i_rCuBG])-5:int(tube_pos[1, i_rCu, i_rCuBG])+5,
                                     :, :, i_rCu, i_rCuBG],  
                      axis=(0,1))
        _tmp = (_tmp[:,0]/np.sum(_tmp[:,0])+_tmp[:,3]/np.sum(_tmp[:,3]))/2
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[1].plot(_nm, _tmp, "-",
                         color=cmap(rCusBG[i_rCuBG]/100.), 
                         label=(
                                "$\mathrm{rCu_{est}^{LU}}=$"
                                +str(np.round(100*fit.x[0],0))+"%"),
                         alpha=1)
        _tmp = np.max(mean_test_corr[int(tube_pos[0, i_rCu, i_rCuBG])-9:int(tube_pos[0, i_rCu, i_rCuBG])+9,
                                     int(tube_pos[1, i_rCu, i_rCuBG])-5:int(tube_pos[1, i_rCu, i_rCuBG])+5,
                                     :, :, i_rCu, i_rCuBG],  
                      axis=(0,1))
        _tmp = (_tmp[:,1]/np.sum(_tmp[:,1])+_tmp[:,2]/np.sum(_tmp[:,2]))/2
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[1].plot(_nm, _tmp, ":",
                         color=cmap(rCusBG[i_rCuBG]/100.), 
                         label=(
                                "$\mathrm{rCu_{est}^{LU}}=$"
                                +str(np.round(100*fit.x[0],0))+"%"),
                         alpha=1)
        
        
    ax[1].set_xlim(680,980)
    ax[1].set_ylim(0,0.115)
    ax[1].set_xlabel("$\lambda$ [nm]")
    ax[0].set_ylabel("$\mathbf{\hat{S}}$ of $\mathrm{rCu_{tube}}$ = "+str(rCus[i_rCu])+"% ")
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['left'].set_linestyle("--")
    ax[1].spines['left'].set_linewidth(1)
    
    ax[1].set_yticks([0,0.1])
    ax[1].set_title("example from validation set (phantom)")
    ax[1].legend(fontsize=7,frameon=False)
    

for i_rCu, rCu in enumerate([1.0]):
    i_rCu = 4
    for rCuBG in [0.0,1.0]:
        df_tmp = df_in_silico[df_in_silico['label_rCu'].isin([rCu])]
        df_tmp = df_tmp[df_tmp['rCu_bg'].isin([rCuBG])]
        df_tmp = df_tmp.reset_index(drop=True)
        i_ill = 0
        _tmp = np.squeeze(np.asarray(np.matrix(df_tmp['descriptor'][0])[0,i_ill*16:(i_ill+1)*16]))
                
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[0].plot(_nm, _tmp/np.sum(_tmp, axis=0), "-",
                         color=cmap(rCuBG), alpha=1,
                         label=("I$_0$"))
        
        i_ill = 1
        _tmp = np.squeeze(np.asarray(np.matrix(df_tmp['descriptor'][0])[0,i_ill*16:(i_ill+1)*16]))
        
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[0].plot(_nm, _tmp/np.sum(_tmp, axis=0), ":",
                         color=cmap(rCuBG), alpha=1,
                         label=("I$_1$"))
        
    ax[0].set_title("example from training set ($\it{in}$ $\it{silico}$)")
    ax[0].set_xlim(680,980)
    ax[0].set_ylim(0,0.115)
    ax[0].set_xlabel("$\lambda$ [nm]")
    ax[0].set_yticks([0,0.1])
    
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['left'].set_linestyle("--")
    ax[0].spines['left'].set_linewidth(1)
    ax[0].legend(fontsize=7,frameon=False)

fig.tight_layout()
fig.show
fig.savefig("_mi_mission.png")
fig.savefig("_mi_mission.svg")


In [ ]:
fig, ax = plt.subplots(len(rCusBG),5, figsize=(10.5,6.5), dpi=300)
cmap = matplotlib.cm.get_cmap('viridis')
d = 10

matplotlib.rcParams.update({'font.size': 7})
matplotlib.rc('xtick', labelsize=7) 
matplotlib.rc('ytick', labelsize=7)

for i_rCu in range(len(rCus)):
    for i_rCuBG, rCuBG in enumerate([0.0,0.25,0.5,0.75,1.0]):
        _tmp = np.max(mean_test_corr[int(tube_pos[0, i_rCu, i_rCuBG])-9:int(tube_pos[0, i_rCu, i_rCuBG])+9,
                                     int(tube_pos[1, i_rCu, i_rCuBG])-5:int(tube_pos[1, i_rCu, i_rCuBG])+5,
                                     :, :, i_rCu, i_rCuBG],  
                      axis=(0,1))
        _tmp = np.sum(_tmp,axis=1)
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[i_rCu,i_rCuBG].plot(_nm, (_tmp/np.sum(_tmp, axis=0)), "+",
                         color=cmap(rCus[i_rCu]/100.), 
                         label=("$\mathrm{rCu_{est}^{LU}}=$"
                                +str(np.round(100*fit.x[0],0))+"%"),
                         alpha=1)
        
        ax[i_rCu,i_rCuBG].set_xlim(675,985)
        ax[i_rCu,i_rCuBG].set_ylim(0,0.2)
        if i_rCu == 4:
            ax[i_rCu,i_rCuBG].set_xlabel("$\lambda$ [nm]")
        else:
            ax[i_rCu,i_rCuBG].set_xlabel(None)
            
        if i_rCuBG == 0:
            ax[i_rCu,i_rCuBG].set_ylabel("$\mathbf{\hat{S}}$")
        else:
            ax[i_rCu,i_rCuBG].set_ylabel(None)
        ax[i_rCu,i_rCuBG].spines['top'].set_visible(False)
        ax[i_rCu,i_rCuBG].spines['right'].set_visible(False)
        ax[i_rCu,i_rCuBG].spines['left'].set_linestyle("--")
        ax[i_rCu,i_rCuBG].spines['left'].set_linewidth(1)
        ax[i_rCu,i_rCuBG].legend(fontsize=7,frameon=False)
    

for i_rCu, rCu in enumerate([0.0,0.25,0.5,0.75,1.0]):
    for i_rCuBG, rCuBG in enumerate([0.0,0.25,0.5,0.75,1.0]):
        df_tmp = df_in_silico[df_in_silico['label_rCu'].isin([rCu])]
        df_tmp = df_tmp[df_tmp['rCu_bg'].isin([rCuBG])]
        df_tmp = df_tmp.reset_index(drop=True)
        i_ill = 0
        _tmp = np.squeeze(np.asarray(np.matrix(df_tmp['descriptor'][0])[0,i_ill*16:(i_ill+1)*16]))
        i_ill = 1
        _tmp = _tmp + np.squeeze(np.asarray(np.matrix(df_tmp['descriptor'][0])[0,i_ill*16:(i_ill+1)*16]))
        
        fit = least_squares(res, x0, args=(_nm, _tmp))
        ax[i_rCu,i_rCuBG].plot(_nm, _tmp/np.sum(_tmp, axis=0), "-",
                         color=cmap(rCu), alpha=1,
                         label=("MC "+
                                "$\mathrm{rCu_{est}^{LU}}=$"
                                +str(np.round(100*fit.x[0],0))+"%"))
        
        if i_rCu == 0:
            ax[i_rCu,i_rCuBG].set_title("$\mathrm{rCu_{bg}}$ = "+str(rCuBG*100)+"%")
        else:
            ax[i_rCu,i_rCuBG].set_title("")
        ax[i_rCu,i_rCuBG].set_yticks([0,0.1])
        ax[i_rCu,i_rCuBG].spines['top'].set_visible(False)
        ax[i_rCu,i_rCuBG].spines['right'].set_visible(False)
        ax[i_rCu,i_rCuBG].spines['left'].set_linestyle("--")
        ax[i_rCu,i_rCuBG].spines['left'].set_linewidth(1)
        ax[i_rCu,i_rCuBG].legend(fontsize=7,frameon=False)

fig.tight_layout()
fig.show
fig.savefig("_mission_direct_compare.png")
fig.savefig("_mission_direct_compare.svg")
